# Simulating interviews with prior survey respondents
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for creating AI agents representing respondents of your existing survey data, and automating simulated follow-on interviews with them.

EDSL is an open-source libary for simulating surveys experiments with AI agents and language models. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## Importing data
We start by importing a dataset of responses from an existing survey. For purposes of demonstration, we use a set of mock responses to a survey about a home construction newsletter that included the following questions:

In [1]:
import pandas as pd

df = pd.read_csv("newsletter_survey_responses.csv")

In [2]:
df[["question_type", "question_text", "question_options"]].drop_duplicates()

,question_type,question_text,question_options
0,Multiple Choice,How often do you read the newsletter?,"Daily, Weekly, Monthly, Rarely, Never"
5,Free Text,What topics would you like to see covered in f...,NaN
10,Linear Scale,"On a scale of 1 to 10, how would you rate the ...",1-10
15,Multiple Choice,Which section of the newsletter do you find mo...,"Construction Tips, Product Reviews, Case Studi..."
20,Free Text,What improvements would you suggest for the ne...,NaN


A sample of the responses:

In [3]:
df[["question_text", "response"]].sample(8)

,question_text,response
12,"On a scale of 1 to 10, how would you rate the ...",9
7,What topics would you like to see covered in f...,Case studies on innovative construction techni...
20,What improvements would you suggest for the ne...,"Adding more video content, such as tutorials a..."
0,How often do you read the newsletter?,Weekly
13,"On a scale of 1 to 10, how would you rate the ...",8
11,"On a scale of 1 to 10, how would you rate the ...",7
14,"On a scale of 1 to 10, how would you rate the ...",7
18,Which section of the newsletter do you find mo...,Construction Tips


## Creating agents for interview subjects
Next we create an `Agent` for each of the respondents and give it information about its responses to the original survey. We do this by turning each respondent's responses into a ditionary of `traits` that we pass to its agent, together with some background information and instructions about the follow-on interview task.

### Designing agent traits

In [4]:
# Change the set of all responses into a dictionary
all_responses = df.to_dict(orient="records")

# Initialize a dictionary to store responses by respondent_id
respondents_dict = {}

# Iterate over each row in the responses
for row in all_responses:
    respondent_id = row["respondent_id"]
    question_id = row["question_id"]
    question_text = row["question_text"]
    question_options = row["question_options"]
    response = row["response"]

    # Format each response as background information for the relevant respondent
    formatted_string = (
        f"You were asked: '{question_text}' "
        f"The answer options were: '{question_options}'. "
        f"You responded: '{response}'"
    )

    # Add the information to the respondent's dictionary
    if respondent_id not in respondents_dict:
        respondents_dict[respondent_id] = {}

    respondents_dict[respondent_id][question_id] = formatted_string

# Print the new dictionary
# print(respondents_dict)

### Adding instructions

In [5]:
# Importing the EDSL class for constructing AI agents
from edsl import Agent

import textwrap
from rich import print

# Interview topic
interview_topic = "home construction"
interview_title = "Home Construction Newsletter Follow-up Interview"

# Persona for the interview subjects
interview_subject_persona = textwrap.dedent(
    f"""\
You are a professional with a keen interest in {interview_topic}.
"""
)

# Instructions for the interview subject agents
interview_subject_instructions = textwrap.dedent(
    f"""\
You recently completed a reader survey about the newsletter of a 
{interview_topic} expert who also produces a popular blog and podcast. 
Now they are asking you some follow-on questions.
"""
)

# Persona for the interviewer agent
interviewer_persona = textwrap.dedent(
    f"""\
You are an well-known expert on {interview_topic} who produces 
a popular newsletter, blog and podcast on {interview_topic}.  
"""
)

# Instructions for the interviewer agent
interviewer_instructions = textwrap.dedent(
    f"""\
You recently conducted a reader survey about your newsletter on
{interview_topic}. Now you are asking respondents some follow-on questions.
"""
)

# Total number of questions to ask in the interview
total_questions = 5

### Constructing the agents

In [6]:
# Initialize a list to store the agents
interview_subjects = []

# Iterate over the respondents' data
for respondent_id, questions in respondents_dict.items():

    # Initialize the traits for each agent
    traits = {}

    # Iterate over the questions
    for question_id, formatted_string in questions.items():
        traits[f"question_id_{question_id}"] = formatted_string

    # Create the agent and add it to the agents list
    agent = Agent(
        name=f"Respondent {respondent_id}",
        traits=traits,
        instruction=interview_subject_instructions,
    )
    interview_subjects.append(agent)

# Inspecting the first one
interview_subjects[0]

Agent(name = 'Respondent 1', traits = {'question_id_1': "You were asked: 'How often do you read the newsletter?' The answer options were: 'Daily, Weekly, Monthly, Rarely, Never'. You responded: 'Weekly'", 'question_id_2': "You were asked: 'What topics would you like to see covered in future issues?' The answer options were: 'nan'. You responded: 'I would love to see more in-depth guides on sustainable building practices. Including case studies from different types of projects would be very helpful. Interviews with experts about their experiences, tips, and challenges they faced would also be appreciated. Additionally, insights into cost-saving techniques would be very beneficial.'", 'question_id_3': "You were asked: 'On a scale of 1 to 10, how would you rate the overall quality of the newsletter?' The answer options were: '1-10'. You responded: '8'", 'question_id_4': "You were asked: 'Which section of the newsletter do you find most valuable?' The answer options were: 'Construction Tips, Product Reviews, Case Studies, Industry News'. You responded: 'Construction Tips'", 'question_id_5': "You were asked: 'What improvements would you suggest for the newsletter?' The answer options were: 'nan'. You responded: 'Adding more video content, such as tutorials and walk-throughs, would make the newsletter more engaging. Summaries of key points at the end of each issue would also be beneficial. It would also be great to include downloadable templates and checklists for various construction projects.'"}, instruction = 'You recently completed a reader survey about the newsletter of a 
home construction expert who also produces a popular blog and podcast. 
Now they are asking you some follow-on questions.
')

## Creating an interviewer agent
We also create an agent for the interviewer using the relevant information and instructions that we've specified above:

In [7]:
# Create an agent for the interviewer
interviewer = Agent(
    traits={"persona": interviewer_persona}, instruction=interviewer_instructions
)

## Conducting the interviews
Here we use EDSL to create some methods for automating an interview between a respondent agent and the interviewer agent. The interview is designed as series of questions presented to the agents with information about what has already transpired. We optionally specify that GPT 4 should be used to generate the content. (Learn more about [selecting language models](https://docs.expectedparrot.com/en/latest/language_models.html) to use with EDSL surveys.)

In [8]:
# Importing the EDSL tools that we will use to administer the questions
from edsl.questions import QuestionFreeText
from edsl import Scenario, Model, Survey

# Selecting a language model to use
# Model.available()  # To see a list of all available models
model = Model("gpt-4")


def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario(
        {"subject": str(subject.traits), "dialog_so_far": dialog_so_far}
    )
    meta_q = QuestionFreeText(
        question_name="next_question",
        question_text="""
        This is the background information for the interview subject: {{ subject }}
        This is your current dialog with the interview subject: {{ dialog_so_far }}
        What question you would ask the interview subject next?
        """,
    )
    question_text = (
        meta_q.by(model)
        .by(researcher)
        .by(scenario)
        .run()
        .select("next_question")
        .first()
    )
    return question_text


def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name="question",
        question_text=f"""
        This is your current dialog with the interview subject: {dialog_so_far}.
        You are now being asked:"""
        + question_text,
    )
    response = q_to_subject.by(model).by(subject).run().select("question").first()
    return response


def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)

    print(" \nQuestion: \n\n" + question_text + "\n\nResponse: \n\n" + response)

    return {"question": question_text, "response": response}


def dialog_to_string(d):
    return "\n".join(
        [f"Question: {d['question']}\nResponse: {d['response']}" for d in d]
    )


def clean_dict(d):
    """Convert dictionary to string and remove braces."""
    return str(d).replace("{", "").replace("}", "")


def summarize_interview(subject, interview_topic, dialog_so_far, researcher):
    interview_subject_name = subject["name"]
    interview_subject_traits = subject["traits"]
    summary_q = QuestionFreeText(
        question_name="summary",
        question_text=(
            f"You have just conducted the following interview of {interview_subject_name} "
            f"who has these traits: {clean_dict(interview_subject_traits)} "
            f"The topic of the interview was {interview_topic}. "
            f"Please draft a summary of the interview: {clean_dict(dialog_so_far)}"
        ),
    )
    themes_q = QuestionFreeText(
        question_name="themes", question_text="List the major themes of the interview."
    )
    survey = Survey([summary_q, themes_q]).set_full_memory_mode()
    results = survey.by(model).by(researcher).run()
    summary = results.select("summary").first()
    themes = results.select("themes").first()
    print("\n\nSummary:\n\n" + summary + "\n\nThemes:\n\n" + themes)


def conduct_interview(subject, researcher, interview_topic):

    print(
        "\n\nInterview subject: "
        + subject["name"]
        + "\n\nInterview topic: "
        + interview_topic
    )

    dialog_so_far = []

    for i in range(total_questions):
        result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
        dialog_so_far.append(result)

    summarize_interview(subject, interview_topic, dialog_so_far, researcher)

In [9]:
for interview_subject in interview_subjects:
    conduct_interview(interview_subject, interviewer, interview_topic)

Interview subject: Respondent 1

Interview topic: home construction

Question: 

Thank you for your detailed feedback. Based on your responses, I understand that you're interested in sustainable 
building practices and cost-saving techniques. Could you please share more about the specific areas of sustainable 
construction you're interested in? Also, could you elaborate on the types of video content and templates you'd find
most useful for your projects?

Response: 

In terms of sustainable construction, I'm particularly interested in energy-efficient building methods, use of 
renewable materials, and waste reduction techniques. It would be great to learn more about the latest innovations 
and trends in these areas. As for the video content, I would find step-by-step tutorials on these sustainable 
practices very useful. Demonstrations of how to use different materials and technologies would also be beneficial. 
Regarding templates, checklists for ensuring energy efficiency and waste reduction during the construction process 
would be very helpful, as well as templates for project planning and cost estimation that incorporate these 
sustainable practices.

Question: 

Thank you for your detailed response. It's great to see your interest in energy-efficient building methods, 
renewable materials, and waste reduction techniques. I'm also glad to hear that step-by-step tutorials and 
demonstrations would be beneficial for you. To better tailor our content to your needs, could you please share some
specific topics or projects you're currently working on or planning to start soon? Additionally, are there any 
specific experts or companies in the field of sustainable construction that you would be interested in hearing from
in our interviews?

Response: 

Currently, I am working on a residential project where we aim to implement energy-efficient building methods. 
Specific topics that I am interested in include insulation techniques for energy efficiency, use of solar panels, 
and rainwater harvesting systems. I would also appreciate content on how to incorporate renewable materials into 
traditional construction practices. As for experts, I would be interested in hearing from professionals like 
William McDonough or companies like LivingHomes that are known for their sustainable construction practices.

Question: 

Thank you for sharing your current project details and specific areas of interest. It's exciting to hear about your
commitment to sustainable practices. Given your interest in insulation techniques for energy efficiency, solar 
panels, and rainwater harvesting systems, would you find it helpful if we included case studies or real-life 
examples of residential projects that have successfully implemented these techniques? Also, would you be interested
in a comparative analysis of different renewable materials and their application in traditional construction? 
Lastly, we can certainly reach out to professionals like William McDonough or companies like LivingHomes for 
interviews. Are there any specific questions or topics you would want us to address in these interviews?

Response: 

Yes, case studies of residential projects that have successfully implemented insulation techniques for energy 
efficiency, solar panels, and rainwater harvesting systems would be extremely helpful. It would provide practical 
insights and tangible examples to learn from. A comparative analysis of different renewable materials and their 
application in traditional construction would also be very beneficial, as it would help in making informed 
decisions for my projects. In the interviews with professionals like William McDonough and companies like 
LivingHomes, I would be interested in learning about their approach to sustainable construction, the challenges 
they face, and their solutions to overcome these challenges. Also, their insights on future trends in sustainable 
construction would be very valuable.

Question: 

Thank you for your thoughtful responses. Your interest in the practical applications of sustainable construction is
evident, and your questions for experts like William McDonough and companies like LivingHomes are insightful. Out 
of curiosity, have you considered or already implemented any sustainable practices in your past projects? If so, 
could you share some of the challenges you faced and how you overcame them? Additionally, are there any other 
resources, tools, or features that you think would enhance the value of our newsletter for you and other readers?

Response: 

In my past projects, I have indeed implemented some sustainable practices. For instance, I have used recycled 
materials where possible and installed energy-efficient windows. One of the challenges I faced was sourcing 
high-quality recycled materials. I overcame this by building relationships with suppliers who specialize in green 
construction materials. Another challenge was convincing clients about the long-term benefits of investing in 
energy-efficient features, given their higher upfront cost. I addressed this by providing them with detailed 
cost-benefit analyses. As for enhancing the value of the newsletter, I think including a section on 'Sustainable 
Construction FAQs' would be beneficial. This could address common challenges and misconceptions about green 
building. Also, a regular feature on 'Green Product of the Month' could introduce readers to new sustainable 
materials and technologies in the market. Finally, a forum or community space where readers can share their 
experiences and ask questions would make the newsletter more interactive and engaging.

...

Question: 

Thank you for your detailed response. It's really helpful. Regarding smart home devices and renewable energy 
technologies, are there any specific brands or models you're interested in? Also, could you tell us what you like 
specifically about the design of 'Dwell' magazine and 'The Verge' newsletter? This will help us understand better 
what improvements we can make to our newsletter.

Response: 

In terms of smart home devices, I am interested in brands like Nest, Ring, Philips Hue, and ecobee. For renewable 
energy technologies, Tesla's solar panels and Powerwall, and Dandelion's geothermal systems have caught my 
attention. Regarding the design of 'Dwell' magazine, I appreciate its clean lines, ample white space, and 
high-quality images that make it easy to focus on the content. The 'The Verge' newsletter is great because of its 
interactive elements like embedded videos and clickable links, as well as its concise presentation of information. 
Both of these publications have a balance between text and visuals that makes them engaging and easy to read.

Question: 

Thank you for sharing your preferences. It's clear that you value both aesthetics and functionality. Based on your 
interest in brands like Nest, Ring, Philips Hue, ecobee, Tesla, and Dandelion, would you like to see detailed 
product reviews, comparisons, and tutorials for these specific brands in our newsletter? Also, considering the 
design elements you liked in 'Dwell' and 'The Verge', would you prefer to see more high-quality images, interactive
elements like videos and links, and a balance of text and visuals in our newsletter? Finally, are there any other 
suggestions you would like to make to further enhance your reading experience?

Response: 

Yes, I would definitely appreciate detailed product reviews, comparisons, and tutorials for brands like Nest, Ring,
Philips Hue, ecobee, Tesla, and Dandelion. It would be great to understand the unique features, installation 
process, and user experience of each product. As for the design, incorporating more high-quality images, videos, 
clickable links, and a balanced mix of text and visuals would certainly enhance the newsletter's appeal. I also 
suggest including a 'Reader's Corner' where subscribers can share their own tips, experiences, or ask questions. 
This could foster a more interactive and engaging community around the newsletter.

Question: 

Thank you for your valuable suggestions. We will definitely consider adding more interactive elements like a 
'Reader's Corner' to foster community engagement. In terms of content, would you prefer a deep dive into one 
product per issue or a comparison of similar products from different brands? Also, would you like to see more 
content from industry experts or user experiences? Lastly, do you have any preferences for the frequency of the 
newsletter or the length of each issue?

Response: 

I would prefer a combination of both deep dives into individual products and comparisons of similar products from 
different brands. This would provide a comprehensive understanding of each product as well as how they stack up 
against each other. As for content, a balance between industry expert insights and real user experiences would be 
ideal, as both perspectives are valuable. Regarding the frequency of the newsletter, I believe a monthly issue is 
sufficient, but each issue could be a bit longer to accommodate the additional content and features.

Question: 

Thank you for your detailed feedback. We truly value your input. Given your interest in both deep dives and 
comparative studies, would you like these to be presented in separate sections or interwoven throughout the issue? 
Also, considering your interest in a balance of expert insights and real user experiences, would you appreciate a 
section dedicated to user stories or testimonials? Finally, in regard to the length of the newsletter, do you have 
a specific range in mind, or do you prefer us to determine the length based on the content of each issue?

Response: 

I think it would be best to interweave the deep dives and comparative studies throughout the issue. This approach 
would provide a more cohesive reading experience. A section dedicated to user stories or testimonials is a great 
idea. It would add a personal touch and provide real-world insights into the use of the products. As for the length
of the newsletter, I don't have a specific range in mind. I believe the length should be determined by the content 
of each issue, ensuring that all topics are covered thoroughly without making the newsletter overly long and 
tedious to read. A balance between comprehensive coverage and readability is key.

Summary:

Respondent 5 reads our newsletter monthly and rates its quality as 7 out of 10. They find the 'Product Reviews' 
section most valuable and suggested improvements in the layout and design, including more visuals and interactive 
elements. They expressed interest in advanced home automation techniques and renewable energy solutions, 
specifically mentioning brands like Nest, Ring, Philips Hue, ecobee, Tesla, and Dandelion. They would like to see 
detailed product reviews, comparisons, and tutorials for these brands. They also suggested a mix of deep dives into
individual products and comparisons of similar products from different brands. They appreciate a balance between 
industry expert insights and real user experiences, and suggested a 'Reader's Corner' for community engagement. 
They are happy with a monthly issue but suggested that each issue could be a bit longer to accommodate the 
additional content and features. They prefer the length to be determined by the content of each issue, ensuring 
comprehensive coverage without sacrificing readability.

Themes:

Respondent 5 reads our newsletter monthly and rates its quality as 7 out of 10. They find the 'Product Reviews' 
section most valuable and suggested improvements in the layout and design, including more visuals and interactive 
elements. They expressed interest in advanced home automation techniques and renewable energy solutions, 
specifically mentioning brands like Nest, Ring, Philips Hue, ecobee, Tesla, and Dandelion. They would like to see 
detailed product reviews, comparisons, and tutorials for these brands. They also suggested a mix of deep dives into
individual products and comparisons of similar products from different brands. They appreciate a balance between 
industry expert insights and real user experiences, and suggested a 'Reader's Corner' for community engagement. 
They are happy with a monthly issue but suggested that each issue could be a bit longer to accommodate the 
additional content and features. They prefer the length to be determined by the content of each issue, ensuring 
comprehensive coverage without sacrificing readability.